# 🔧 FunctionGemma Quick Test - Fixed Prompt

This tests the model WITHOUT the X,Y,Z placeholder issue.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install -q transformers peft torch accelerate bitsandbytes huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import torch
import torch.distributed as dist
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import re

try:
    if not dist.is_initialized():
        dist.init_process_group(backend="gloo", init_method="file:///tmp/fix_test", rank=0, world_size=1)
except: pass

BASE_MODEL_ID = "google/medgemma-4b-it"
ADAPTER_ID = "NurseCitizenDeveloper/nursing-function-gemma"

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_ID, quantization_config=bnb_config, device_map={"": 0}, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = PeftModel.from_pretrained(model, ADAPTER_ID)
print("✅ Model loaded!")

In [ ]:
# FIXED PROMPT - No X,Y,Z placeholders!
def test_with_fixed_prompt(user_input):
    # Simple, clean prompt - just describe what to do
    prompt = f"""<start_of_turn>user
You are a clinical AI agent. Convert this clinical note into a function call.

Available functions:
- record_vitals: Records blood pressure, heart rate, temperature
- administer_medication: Logs medication given
- search_nmc_standards: Searches nursing guidelines

Input: {user_input}

Output the function call with the actual values from the input.<end_of_turn>
<start_of_turn>model
"""
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, do_sample=False)
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "<start_of_turn>model" in response:
        response = response.split("<start_of_turn>model")[-1].strip()
    if "<end_of_turn>" in response:
        response = response.split("<end_of_turn>")[0].strip()
    return response

# Test cases
tests = [
    "BP is 120/80, pulse 72",
    "Gave Paracetamol 500mg orally",
    "What does NMC say about confidentiality?"
]

print("🧪 Testing with FIXED prompt (no placeholders):")
print("="*60)
for t in tests:
    result = test_with_fixed_prompt(t)
    print(f"\nInput: {t}")
    print(f"Output: {result}")

In [ ]:
# Try even simpler - just ask directly
def test_simple(user_input):
    prompt = f"""<start_of_turn>user
Convert to function call: {user_input}<end_of_turn>
<start_of_turn>model
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, do_sample=False)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "<start_of_turn>model" in response:
        response = response.split("<start_of_turn>model")[-1].strip()
    return response

print("\n\n🧪 Testing with SIMPLE prompt:")
print("="*60)
for t in tests:
    result = test_simple(t)
    print(f"\nInput: {t}")
    print(f"Output: {result}")